In [19]:
import pandas as pd, numpy as np
from scipy.stats import boxcox

# Intro

Notebook takes observed and predicted lake chemistry/ecol values output by notebook 02_BN_development_1Season (from the continuous network cross validation section). These are then discretized into WFD-related boundaries, and classification error is calculated. This classification error can then be compared to the classification error obtained by cross validation of discrete networks using the same classifications, or just to provide supporting error info to accompany predictions of probabilities of being in a certain class.

# Set up for processing

In [20]:
# Set up for processing below

# Boundaries dictionary, copied from notebook B_seasonal_data_matrix_1Season
bound_dict = {
             'TP': [29.5], # No data below 20, so drop this class boundary. 29.5 is middle of 'Mod' class   
             'chla': [20.0],  # WFD boundaries: [10.5, 20.0]. But only 6 d.p. under 10.5 so merge G and M classes.
                              # For predicting cyano, would be better 17.4.   
             'colour': [51.2], # [51.2] if comparing CV scores of continuous and discrete. [48.0] if calculating CV score of final network (665h percentile)
             'cyano': [1.0], # M-P boundary is 2.0, but there were only 2 values in this class. Plenty above 2 tho.
             }

def discretize(thresholds, value):
    """
    Function to compare a number to a list of thresholds and categorise accordingly. E.g. to apply row-wise down a df
    Input:
        thresholds: list of boundaries that define classes of interests
        value: float to be compared to the thresholds
    Returns: class the value lies in. Classes are defined in factor_li_dict within function according to number of thresholds
    """
    
    if np.isnan(value):
        return np.NaN
    
    factor_li_dict = {2: ['0','1'],
                     3: ['0','1','2'],}
    
    n_classes = len(thresholds)+1
    
    for i, boundary in enumerate(thresholds):
    
        if value<boundary:
            return factor_li_dict[n_classes][i]
            break # Break out of loop
        
        # If we're up to the last class boundary, and the value is bigger than it, value is in the uppermost class
        if i+1 == len(thresholds) and value >= boundary:
            return factor_li_dict[n_classes][i+1]
        
def classification_error(df):
    """
    Calculate classification error, the proportion of time the modle predicted the class correctly
    
    Input:
        df: dataframe with two columns, 'obs' and 'pred'
    Output:
        Classification error (float)
    """

    # Were observed and predictions in the same class? 'right' col is a boolean, 1=Yes the same, 0=no different
    right = np.where((df['obs'] == df['pred']), 1, 0)

    classification_error = 1 - (right.sum() / len(right))
    return classification_error


In [21]:
def xval_postprocess(var, fpath):
    """
    Function to read in a csv of observed and predicted values from a continuous
    Bayesian belief network produced in BNLearn R notebook, and calculate correlation
    coefficients, and then classify according to WFD and work out classification error
    for comparison to classification error calculated directly in BNLearn using discrete network.
    
    Inputs:
        var: string, one of 'TP','chla','cyano','colour_summer'
        fpath: string giving location of csv to be read in. csv should have columns:
                'obs_1','pred_1','obs_2','pred_2',... where _1, _2, etc. is the cross
                validation run number.
    Returns:
    Printed output, plus dictionary of results, with keys
        'corr_coeffs': series of correlation coefficients, one value per cross validation run
        'classification_errors': series of classification errors, one value per cross validation run
        'cont_data_dict': dict of observed and predicted dfs, continuous data, one df per xval run
        'classified_data_dict': dict of observed and predicted dfs, classified data, one df per xval run
        
    """

    # ---------------------------------------------------------------------------------
    # Read in data
    df = pd.read_csv(fpath, index_col=0)

    # ---------------------------------------------------------------------------------
    # Split into separate dataframes for each cross validation run
    cont_dict = {}  # Key: run number, returns df with obs and pred
    for i, col_name in enumerate(df.columns):
        if i % 2 == 0:  # If even, i.e. only do this for half the cols
            run_no = int(col_name.split("_", 1)[1])
            if run_no == 1:
                temp_df = df.iloc[:, [0, 1]]
            else:
                temp_df = df.iloc[:, [2 * run_no - 2, 2 * run_no - 1]]

            temp_df.columns = ["obs", "pred"]
        cont_dict[run_no] = temp_df

    # ---------------------------------------------------------------------------------
    # Calculate correlation coefficients, convert to WFD classes and work out classification error

    cc_dict = {}  # key: run_no, returns correlation coeff
    mse_dict = {}  # key: run_no, returns mse (pred - obs)
    classified_dict = {} # Key: run number. Returns df with cols 'obs' and 'pred', discrete data
    class_error_dict = {}

    for run_no in cont_dict.keys():

        cont_df = cont_dict[run_no]  # df with continuous data

        # Correlation coefficients
        cc_dict[run_no] = cont_df["obs"].corr(cont_df["pred"], method="pearson")

        # Mean square error
        mse = np.mean(((cont_df["pred"] - cont_df["obs"]) ** 2))
        mse_dict[run_no] = mse

        # Classify obs and pred into WFD (or related) class boundaries
        disc_df = pd.DataFrame(index=cont_df.index, columns=cont_df.columns)  # New empty df to be populated
        for col in cont_df.columns:
            disc_df[col] = cont_df[col].apply(lambda x: discretize(bound_dict[var], x))
        classified_dict[run_no] = disc_df

        # Calculate classification error (proportion of time model predicted class correctly)
        error = classification_error(disc_df)
        class_error_dict[run_no] = error

    # ---------------------------------------------------------------------------------
    # Aggregate results over runs

    corr_coeffs = pd.Series(cc_dict)  # These match those calculated by bnlearn, good!
    mses = pd.Series(mse_dict)
    errors = pd.Series(class_error_dict)

    # ---------------------------------------------------------------------------------
    # Take a look at the output
#     print("Correlation coefficients, %s:" % var)
#     print(corr_coeffs)
    print("\nMean correlation coefficient, %s: %s" % (var, corr_coeffs.mean()))

#     print("\nmse, %s:" % var)
#     print(mses)
    print("Mean mse, %s: %s" % (var, mses.mean()))

#     print("\nClassification errors, %s:" % var)
#     print(errors)
    print("Mean classification error for %s: %s" % (var, errors.mean()))

    # Return dictionaries
    results_dict = {
                    "corr_coeffs": corr_coeffs,
                    "classification_errors": errors,
                    "cont_data_dict": cont_dict,
                    "classified_data_dict": classified_dict,
                    }
    return results_dict

# TP

In [22]:
# With wind, all nodes
fpath = "../Data/CrossValidation/TP_continuous_fromAllNodes.csv"
results_dict = xval_postprocess("TP", fpath)


Mean correlation coefficient, TP: 0.7181037536758927
Mean mse, TP: 11.185997265243895
Mean classification error for TP: 0.3210526315789474


In [23]:
# With wind, predictable nodes
fpath = "../Data/CrossValidation/TP_continuous_fromPredictableNodes.csv"
results_dict = xval_postprocess("TP", fpath)


Mean correlation coefficient, TP: 0.5759801439422972
Mean mse, TP: 15.681293255289784
Mean classification error for TP: 0.3289473684210526


In [24]:
# Remove wind, all nodes
fpath = "../Data/CrossValidation/TP_cont_fromAllNodes_noWind.csv"
results_dict = xval_postprocess("TP", fpath)


Mean correlation coefficient, TP: 0.7325657242987758
Mean mse, TP: 10.76175599303794
Mean classification error for TP: 0.3078947368421053


In [25]:
# Remove wind, predictable nodes only
fpath = "../Data/CrossValidation/TP_cont_fromPredictableNodes_noWind.csv"
results_dict = xval_postprocess("TP", fpath)


Mean correlation coefficient, TP: 0.5792585066304219
Mean mse, TP: 15.545155522656668
Mean classification error for TP: 0.33421052631578946


## TP: Comparison of classification errors between continuous and discrete networks:

Including met.no wind:
- Predictable/measurable nodes, discrete BN: 0.55
- Predictable/measurable nodes, continuous BN: 0.36
- All nodes, discrete BN: 0.59
- All nodes, continuous BN: 0.32

**Other structures**

Remove met.no wind:
- All stats improved a bit without wind in continuous network. Therefore **decide to remove wind-TP link**.
- **Comparison of discrete and continuous network classification error, without metno wind**, excluding 2019 data, becomes:

    - Predictable/measurable nodes, discrete: 0.51
    - Predictable/measurable nodes, continuous: 0.33
    - All nodes, discrete: 0.53
    - All nodes, continuous: 0.31
    
So take this as the final comparison of discrete vs continuous for TP, and part of the justification for going continuous. Other justification is the cpts in the discrete, which don't show the right behaviour (see markdown comments in BN development notebook).

# Chl-a

In [26]:
fpath = "../Data/CrossValidation/chla_continuous_fromPredictableNodes.csv"
results_dict = xval_postprocess("chla", fpath)


Mean correlation coefficient, chla: 0.5349801355638153
Mean mse, chla: 22.97000935631268
Mean classification error for chla: 0.3157894736842105


In [27]:
fpath = "../Data/CrossValidation/chla_continuous_fromAllNodes.csv"
results_dict = xval_postprocess("chla", fpath)


Mean correlation coefficient, chla: 0.7123435737234363
Mean mse, chla: 15.572902979921256
Mean classification error for chla: 0.24210526315789474


In [28]:
# Remove wind, predictable nodes
fpath = "../Data/CrossValidation/chla_cont_fromPredictableNodes_noWind.csv"
results_dict = xval_postprocess("chla", fpath)


Mean correlation coefficient, chla: 0.5387591347925305
Mean mse, chla: 22.790799773849056
Mean classification error for chla: 0.3157894736842105


In [29]:
# Remove wind, all nodes
fpath = "../Data/CrossValidation/chla_cont_fromAllNodes_noWind.csv"
results_dict = xval_postprocess("chla", fpath)


Mean correlation coefficient, chla: 0.7082228333391504
Mean mse, chla: 15.675156887827393
Mean classification error for chla: 0.21315789473684207


## Chl-a, comparison of classification error between continuous and discrete

Whole network used to predict chl-a node:
- discrete BN:0.08
- continuous BN: 0.21
- continuous, adding extra WFD class (G-M boundary at 10.0): 0.44

Only include nodes that will be updated when forecasting:
- discrete BN: 0.08
- continuous BN: 0.32
- continuous, adding extra WFD class (G-M boundary at 10.0): 0.49

Lower classification error when using the discrete network. But problems with the discrete chla cpts outweigh these differences:
- When chla_prevSummer is high, changing wind speed from L to H doesn't result in any change to probs for low TP. But if TP is high, higher wind speed makes it more likely to have high chla. This isn't right, and is only because of a lack of data points. Therefore would have had to remove the wind-chla link.
- Even after doing this, still have probs: when previous summer's chl-a is low, chla is always predicted to be low, and TP has no effect. When previous summer's chl-a is high, the TP effect is the wrong way around: when TP is low, expect high chla. When TP is high, have a lower chance of high TP than when TP is low. Again, just due to low data volume

## Other structures

- Removing wind: no change or slight deterioration, depending on whether you predict using just predictable nodes (no change) or the whole network (slight deterioration).

# Cyano

First, need to alter the boundaries in the boundaries dict for cyano, to take account of the box-cox transformation applied to the continuous data:  y* = (y^L - 1)/L, where we used lambda=0.1 when transforming original cyano data.

In [34]:
bound_dict['cyano'] = boxcox(bound_dict['cyano'], lmbda=0.1)
bound_dict['cyano']

array([0.])

In [35]:
fpath = "../Data/CrossValidation/cyano_continuous_fromPredictableNodes.csv"
results_dict = xval_postprocess("cyano", fpath)


Mean correlation coefficient, cyano: 0.6295420438153104
Mean mse, cyano: 1.0102731490753691
Mean classification error for cyano: 0.16521739130434784


In [36]:
fpath = "../Data/CrossValidation/cyano_continuous_fromAllNodes.csv"
results_dict = xval_postprocess("cyano", fpath)


Mean correlation coefficient, cyano: 0.7839995687523232
Mean mse, cyano: 0.6510882952764112
Mean classification error for cyano: 0.1782608695652174


## Comparison of classification error, continuous vs discrete networks:

- Predictable/measurable nodes, discrete: 0.23
- Predictable/measurable nodes, continuous: 0.17
- All nodes, discrete: 0.13
- All nodes, continuous: 0.18

In this case, the continuous network has lower classification error than the discrete network when using predictable/measurable nodes, but slightly higher error when using all nodes (first time this has been the case). But errors still pretty low, and added benefit of the relationships being sensible between nodes rather than the cpt issue we have for one of the cpts in the cyano class: in the high summer colour class, as chla increases from L to H, the chance of high cyano decreases. I think we would expect positive relationship between chla and cyano regardless of colour, just a lower chance of high cyano when colour is higher. So this is another low data vol artefact.

**To investigate in the future:**
- Cross val of cyano predictive ability with/without colour-cyano link

# Proposed changes to structure based on cross validation results

(in this notebook, and the BN_Development_1Season one)

- Remove wind-TP link. Keep wind-chla link.
- Keep rain-colour link